# Bio668 Notebook Submission
### Import python modules

###### *Note:* If the plots do not display correctly in the notebook, they can be found in the `plots` directory that is created by the script. This can happen depending on how the jupyter notebook client is set up. The plots are `.html` files and can be opened in any browser.

[Readme.md](https://raw.githubusercontent.com/seanfahey1/Bio668/main/README.md)

In [19]:
import pandas as pd
from pathlib import Path
import plotly.express as px
from plotly.io import to_html
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

### Import local functions

In [20]:
from pca_of_kmers import get_kmers, do_pca

Initial setup

In [21]:
Path("plots/").mkdir(exist_ok=True)
Path("sequences/").mkdir(exist_ok=True)

### Get 3-mer array

`get_kmers()` loops through all `.fasta` formatted **protein sequence** files in the `sequences` directory and returns a numpy array of the
relative frequency of each 3-mer in all files, plus lists of the headers of each sequence and the source file of each
sequence in the same order as the rows of the array.

The file list is used both to annotate the PCA pltos, and as the response value in the random forest. The header list
is only used for additional plot annotation.

In [22]:
X, headers, files, kmer_list = get_kmers()

Getting 3-mers..
file: portal_200.fasta	parsing record: 0
file: portal_200.fasta	parsing record: 20
file: portal_200.fasta	parsing record: 40
file: portal_200.fasta	parsing record: 60
file: portal_200.fasta	parsing record: 80
file: portal_200.fasta	parsing record: 100
file: portal_200.fasta	parsing record: 120
file: portal_200.fasta	parsing record: 140
file: portal_200.fasta	parsing record: 160
file: portal_200.fasta	parsing record: 180
file: collar_tail_fiber_200.fasta	parsing record: 0
file: collar_tail_fiber_200.fasta	parsing record: 20
file: collar_tail_fiber_200.fasta	parsing record: 40
file: collar_tail_fiber_200.fasta	parsing record: 60
file: collar_tail_fiber_200.fasta	parsing record: 80
file: collar_tail_fiber_200.fasta	parsing record: 100
file: collar_tail_fiber_200.fasta	parsing record: 120
file: collar_tail_fiber_200.fasta	parsing record: 140
file: collar_tail_fiber_200.fasta	parsing record: 160
file: collar_tail_fiber_200.fasta	parsing record: 180
file: baseplate_200.fasta

### PCA

`do_PCA()` uses `sklearn.decomposition.PCA` to fit a PCA to the array provided. It then produces 3 plots using plotly:
- % Variance Explained per Principal Component
- 2D PCA using the first 2 principal components
- 3D PCA using the first 3 principal components

The PCA plots are grouped and colored by source file for each protein sequence. Each point on the plot corresponds to
a single protein sequence from the input files. The header information from the fasta sequence can be seen for each
point when hovering over the point.

The plots are also saved as .html files that can be opened in the browser.

In [23]:
do_pca(X, headers, files)

Starting PCA..


### Random Forest Model

In [24]:
# train test split for the predictions portion
X_train, X_test, y_train, y_test = train_test_split(
    X,
    files,
    test_size=0.33,
    random_state=987
)

# fit the model
forest = RandomForestClassifier(random_state=987)
forest.fit(X_train, y_train)

# predictions
predictions = forest.predict(X_test)

# check accuracy
accuracy = sum(1 for i, x in enumerate(predictions) if x == y_test[i]) / len(predictions)
print(f'accuracy: {accuracy:.2%}')

accuracy: 89.02%


Accuracy of 89% is pretty good for a default Random Forest model. This means that the random forest feature importances
are going to be a reasonable way to figure out which 3-mers are most important.

### Random Forest Feature Importances

In [25]:
# re-fit a model with no train-test split (we want to see all the data for this one)
forest = RandomForestClassifier(random_state=987)
forest.fit(X, files)

# get the feature importance for each 3-mer and add to a data frame
importances = list(forest.feature_importances_)
importance_df = pd.DataFrame(data={'3-mer': kmer_list, 'RF_feature_importance': importances})

# sort the dataframe and look at the top 5 most important features
importance_df.sort_values(by='RF_feature_importance', ascending=False, inplace=True)
print(importance_df.head())

      3-mer  RF_feature_importance
2656    EPR               0.008337
3534    GDG               0.008066
7876    NRE               0.006134
12833   YGS               0.004734
6472    LFR               0.004434


In [34]:
# plot the top 50 most important features
rf_plot = px.bar(
    importance_df.head(50),
    y='RF_feature_importance',
    x='3-mer'
).update_layout(
    title="<b>Random Forest Feature Importance scores</b><br>for the top 50 most important 3-mers",
    yaxis_title="RF feature importance score",
    yaxis_tickformat=".1%",
)

rf_plot.show()

with open("plots/random_forest_scores.html", "w") as fig_out:
    fig_out.write(to_html(rf_plot, include_plotlyjs="cdn"))